In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

options = Options()
options.add_argument("--headless")  # Run in the background
options.add_argument("--disable-blink-features=AutomationControlled")  # Avoid bot detection

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get("https://www.france24.com/ar/%D8%A3%D8%B1%D8%B4%D9%8A%D9%81/2025/")

html = driver.page_source
time.sleep
driver.quit()
# Parse the page source with BeautifulSoup
soup = BeautifulSoup(html, "html.parser")
soup

<html class="webp webp-alpha webp-animation webp-lossless" dir="rtl" lang="ar"><head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0, viewport-fit=cover" name="viewport"/>
<link href="/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="/apple-touch-icon.png" rel="apple-touch-icon"/>
<link href="/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
<link href="/favicon-194x194.png" rel="icon" sizes="194x194" type="image/png"/>
<link href="/android-chrome-192x192.png" rel="icon" sizes="192x192" type="image/png"/>
<link href="/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>
<link href="/ar/site.manifest" rel="manifest"/>
<link color="#00a7e3" href="/safari-pinned-tab.svg" rel="mask-icon"/>
<title>أرشيف فرانس 24 لسنة 2025: كل المقالات والتسجيلات وفيديوهات الأحداث الدولية</title>
<link href="https://www.france24.com/ar/%D8%A3%D8%B1%D8%B4%D9%8A%D9%81/2025/" rel="canonical"/>
<meta content="https://s.france24.com/media/dis

In [3]:
# Look for containers holding articles, such as divs or sections
links_tag = soup.find_all("a", class_="o-archive-month__days__day__link")  # Replace with actual class name

# Check if it contains any articles
links=[]
dates=[]
for link in links_tag:
    links.append(f"https://www.france24.com{link['href']}")
    dates.append(link.text)
    print(f"https://www.france24.com{link['href']}",link.text)  # Print the HTML of each container
print(len(dates))

https://www.france24.com/ar/%D8%A3%D8%B1%D8%B4%D9%8A%D9%81/2025/04/18-%D8%A3%D8%A8%D8%B1%D9%8A%D9%84-2025 
                18 أبريل 2025
            
https://www.france24.com/ar/%D8%A3%D8%B1%D8%B4%D9%8A%D9%81/2025/04/17-%D8%A3%D8%A8%D8%B1%D9%8A%D9%84-2025 
                17 أبريل 2025
            
https://www.france24.com/ar/%D8%A3%D8%B1%D8%B4%D9%8A%D9%81/2025/04/16-%D8%A3%D8%A8%D8%B1%D9%8A%D9%84-2025 
                16 أبريل 2025
            
https://www.france24.com/ar/%D8%A3%D8%B1%D8%B4%D9%8A%D9%81/2025/04/15-%D8%A3%D8%A8%D8%B1%D9%8A%D9%84-2025 
                15 أبريل 2025
            
https://www.france24.com/ar/%D8%A3%D8%B1%D8%B4%D9%8A%D9%81/2025/04/14-%D8%A3%D8%A8%D8%B1%D9%8A%D9%84-2025 
                14 أبريل 2025
            
https://www.france24.com/ar/%D8%A3%D8%B1%D8%B4%D9%8A%D9%81/2025/04/13-%D8%A3%D8%A8%D8%B1%D9%8A%D9%84-2025 
                13 أبريل 2025
            
https://www.france24.com/ar/%D8%A3%D8%B1%D8%B4%D9%8A%D9%81/2025/04/12-%D8%A3%D8%A8%D8%B1%D9%8A%D9%84

In [5]:
import requests
from bs4 import BeautifulSoup
import csv
def get_data(lien,date):
    try:
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
        driver.get(lien)
        html = driver.page_source
        driver.quit()
        # Parse the page source with BeautifulSoup
        soup1 = BeautifulSoup(html, "html.parser")
        try:
            ds=date.split()
            f=""
            for d in ds:
                f=f+"_"+d
            print(f)
            articles = soup1.find_all("li", class_="o-archive-day__list__entry")
            # Open a CSV file to save the data
            with open(f"articles/france24_arabe_articles_archive_{f}.csv", "w", newline="", encoding="utf-8") as file:
                writer = csv.writer(file)
                # Write the header
                writer.writerow(["source",  "title", "content", "description", "url", "published_at", "tags"])
            
                for article in articles:
                    # Source
                    source = "France 24"  # Since it's from France24
                    # Extract the link to the article
                    link_tag = article.find("a",class_="a-archive-link", href=True)
                    article_link = f"https://www.france24.com{link_tag['href']}" if link_tag else "No Link"
                
                    # Extract the article title
                    title_tag = article.find("h2")
                    title = title_tag.text.strip() if title_tag else "No Title"
                    published_at=date
                    
                    content=""
                    description=""
                    tags=""
                    # Write the extracted data to CSV
                    writer.writerow([source, title, content, description, article_link,  published_at, tags])
            
            print("✅ Data saved")
        except (Exception) as e:
                print(f"Error1: {e}")
    except (Exception) as e:
            print(f"Error2: {e}")

In [7]:
import pandas as pd
import time
def complete_data(date):
    ds=date.split()
    f=""
    for d in ds:
        f=f+"_"+d
    print(f)
    df=pd.read_csv(f"articles/france24_arabe_articles_archive_{f}.csv")
    print(df.shape[0])
    for i in range(df.shape[0]):
        if(df.loc[i,'url']):
            try:
                driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
                driver.get(df.loc[i,'url'])
                html = driver.page_source
                #time.sleep
                driver.quit()
                # Parse the page source with BeautifulSoup
                soup2 = BeautifulSoup(html, "html.parser")
                soup2
    
                print(i,end='')
                # Extract all news headlines
                contents = soup2.find_all("p", class_="")
                cont = ""
                # Print extracted contents
                for content in contents:
                    cont= cont + "\n"+ content.text.strip()
                df.loc[i,'content']=cont
                # Extract all news headlines
                description = soup2.find("p", class_="t-content__chapo")
                df.loc[i, "description"] = description.text if description else None
                
                times = soup2.find("time", class_="")
                df.loc[i, 'published_at'] = times.text if times else None

                tags=soup2.find_all("div", class_="a-tag-section")
                if tags:
                    df.loc[i,'tags'] = tags[0].text
                else:
                    df.loc[i,'tags'] = None  # Ou "" si tu veux une chaîne vide
            except (Exception) as e:
                print(f"Error: {e}")

    df.to_csv(f"data/france24_arabe_articles_content_archive_{f}.csv")
    print("✅ Data saved")
    


In [11]:
for i in range(len(dates)):
    get_data(links[i],dates[i])

_18_أبريل_2025
✅ Data saved
_17_أبريل_2025
✅ Data saved
_16_أبريل_2025
✅ Data saved
_15_أبريل_2025
✅ Data saved
_14_أبريل_2025
✅ Data saved
_13_أبريل_2025
✅ Data saved
_12_أبريل_2025
✅ Data saved
_11_أبريل_2025
✅ Data saved
_10_أبريل_2025
✅ Data saved
_09_أبريل_2025
✅ Data saved
_08_أبريل_2025
✅ Data saved
_07_أبريل_2025
✅ Data saved
_06_أبريل_2025
✅ Data saved
_05_أبريل_2025
✅ Data saved
_04_أبريل_2025
✅ Data saved
_03_أبريل_2025
✅ Data saved
_02_أبريل_2025
✅ Data saved
_01_أبريل_2025
✅ Data saved
_31_مارس_2025
✅ Data saved
_30_مارس_2025
✅ Data saved
_29_مارس_2025
✅ Data saved
_28_مارس_2025
✅ Data saved
_27_مارس_2025
✅ Data saved
_26_مارس_2025
✅ Data saved
_25_مارس_2025
✅ Data saved
_24_مارس_2025
✅ Data saved
_23_مارس_2025
✅ Data saved
_22_مارس_2025
✅ Data saved
_21_مارس_2025
✅ Data saved
_20_مارس_2025
✅ Data saved
_19_مارس_2025
✅ Data saved
_18_مارس_2025
✅ Data saved
_17_مارس_2025
✅ Data saved
_16_مارس_2025
✅ Data saved
_15_مارس_2025
✅ Data saved
_14_مارس_2025
✅ Data saved
_13_مارس_2

In [9]:
for i in range(54,len(dates)):
    print(i)
    complete_data(dates[i])

54
_23_فبراير_2025
17
0

C:\Users\HP\AppData\Local\Temp\ipykernel_11996\1262259936.py:30: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[i,'content']=cont
C:\Users\HP\AppData\Local\Temp\ipykernel_11996\1262259936.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '
                                                                        أعلن الأمين العام لحزب الله اللبناني نعيم قاسم الأحد خلال كلمة ألقاها بمناسبة تشييع سلفه الراحل حسن نصر الله بمدينة كميل شمعون الرياضية في جنوب بيروت، أن "المقاومة لم تنته" وأنها "مستمرة"، في رسالته بهذا الحدث الجماهيري الأول من نوعه منذ المواجهة المفتوحة بين الجماعة المدعومة من إيران وإسرائيل وانتهت بوقف لإطلاق النار في 27 نوفمبر/تشرين الثاني، وخرج منها الحزب ضعيفا سياسيا وعسكريا.                                ' has

1

C:\Users\HP\AppData\Local\Temp\ipykernel_11996\1262259936.py:40: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '

                                    الأخبار المغاربية
                            ' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[i,'tags'] = tags[0].text


2345678910111213141516✅ Data saved
55
_22_فبراير_2025
16
0

...

1234567891011121314151617181920212223242526272829303132✅ Data saved
106
_02_يناير_2025
26
0

C:\Users\HP\AppData\Local\Temp\ipykernel_11996\1262259936.py:30: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[i,'content']=cont
C:\Users\HP\AppData\Local\Temp\ipykernel_11996\1262259936.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '
                                                                        أعلنت شرطة  كوريا الجنوبية أنّها دهمت صباح الخميس مطار موان في جنوب البلاد ومكاتب "جيجو" في إطار تحقيقاتها في الكارثة الجوية التي شهدها الأحد حين تحطّمت فيه طائرة ركاب ما أسفر عن مقتل 179 شخصا. وتجري منذ الساعة التاسعة صباحا من يوم الثاني من كانون الثاني/يناير عملية تفتيش وضبط في ثلاثة مواقع".    هي مطار موان، ومقرّ شركة جيجو للطيران في سيول بالإضافة إلى مكتب لسلطة الطيران الإقليمية في موان.                           

12345678910111213141516171819202122232425✅ Data saved
107
_01_يناير_2025
13
0

C:\Users\HP\AppData\Local\Temp\ipykernel_11996\1262259936.py:30: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '
أعلنت وزارة النقل الكورية الجنوبية الأربعاء بأنها ستقوم بإرسال مسجل بيانات رحلة الطائرة التي تحطمت صباح الأحد إلى الولايات المتحدة لإجراء التحليلات.
وتمكن المحققون من استخراج بيانات أولية من أحد الصندوقين الأسودين للطائرة المنكوبة، وفق ما أعلن مسؤول الأربعاء. ويقوم محققون كوريون جنوبيون وأمريكيون، بعضهم من شركة بوينغ، بتمشيط موقع الحادث في مطار موان الدولي بجنوب غرب البلاد.
في هذا الصدد، قال نائب وزير الطيران المدني جو جونغ-وان: "الاستخراج الأولي لمسجل صوت قمرة القيادة اكتمل"، مشيرا إلى أن الخبراء ما زالوا يحاولون استخراج البيانات من الصندوق الأسود الثاني الذي يسجل بيانات الرحلة.
وتابع جو جونغ-وان: "بناء على هذه البيانات الأولية، نستعد للبدء في تحويلها إلى صيغة صوتية" ما من شأنه السماح للمحققين بالاستماع إلى التبادلات الأخيرة للطيارين. مشيرا إلى أن الصندوق الأسود الثاني، وهو مسجل بيانات الرحلة 

123456789101112✅ Data saved


In [17]:
import os
import pandas as pd

# Chemin du dossier contenant les fichiers CSV
dossier_csv = "data"

# Liste pour stocker les DataFrames
dataframes = []

# Lire tous les fichiers CSV du dossier
for fichier in os.listdir(dossier_csv):
    if fichier.endswith(".csv"):
        chemin_fichier = os.path.join(dossier_csv, fichier)
        df = pd.read_csv(chemin_fichier)
        dataframes.append(df)

# Fusionner tous les DataFrames
df_fusionne = pd.concat(dataframes, ignore_index=True)

# Enregistrer le résultat dans un nouveau fichier CSV
df_fusionne.to_csv("fichier_fusionne_2025_2024.csv", index=False)

print("Fusion terminée ! Fichier enregistré sous 'fichier_fusionne.csv'.")


Fusion terminée ! Fichier enregistré sous 'fichier_fusionne.csv'.


In [18]:
df_fusionne.shape

(10756, 10)

In [23]:
df_fusionne.isnull().sum()

Unnamed: 0          0
source              0
title               0
content          6454
description       341
url                 0
published_at       90
tags              634
author          10456
image_url?      10456
dtype: int64

In [62]:
df_fusionne[df_fusionne['description']=='No Description'].shape

(19, 11)

In [ ]:
df_fu

In [64]:
for li in df_fusionne.loc[df_fusionne.description.isnull(), 'url']:
        if(li):
            try:
                driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
                driver.get(li)
                html = driver.page_source
                driver.quit()
                # Parse the page source with BeautifulSoup
                soup2 = BeautifulSoup(html, "html.parser")
                soup2
    
     
                # Extract all news headlines
                contents = soup2.find_all("p", class_="")
                cont = ""
                # Print extracted contents
                for content in contents:
                    cont= cont + "\n"+ content.text.strip()
                df_fusionne[df_fusionne.url==li].content=cont
                # Extract all news headlines
                description = soup2.find("p", class_="t-content__chapo")
                if description:
                    df_fusionne[df_fusionne.url==li].description=description.text
                times = soup2.find("time", class_="")
                df_fusionne[df_fusionne.url==li].published_at=times.text
                tags=soup2.find_all("div", class_="a-tag-section")
                df_fusionne[df_fusionne.url==li].tags=tags[0].text
            except (Exception) as e:
                print(f"Error: {e}")


https://www.france24.com/ar/%D8%A3%D9%85%D8%B1%D9%8A%D9%83%D8%A7/20250102-%D9%87%D9%84-%D9%84%D9%87-%D8%B9%D9%84%D8%A7%D9%82%D8%A9-%D8%A8%D9%87%D8%AC%D9%88%D9%85-%D9%86%D9%8A%D9%88-%D8%A3%D9%88%D8%B1%D9%84%D9%8A%D8%A7%D9%86%D8%B2-%D8%A7%D9%86%D9%81%D8%AC%D8%A7%D8%B1-%D8%B3%D9%8A%D8%A7%D8%B1%D8%A9-%D8%AA%D8%B3%D9%84%D8%A7-%D8%A3%D9%85%D8%A7%D9%85-%D9%81%D9%86%D8%AF%D9%82-%D8%AA%D8%B1%D8%A7%D9%85%D8%A8-%D9%81%D9%8A-%D9%84%D8%A7%D8%B3-%D9%81%D9%8A%D8%BA%D8%A7%D8%B3-%D9%88%D9%85%D9%82%D8%AA%D9%84-%D8%B3%D8%A7%D8%A6%D9%82%D9%87%D8%A7
https://www.france24.com/ar/%D8%A8%D8%B1%D8%A7%D9%85%D8%AC/%D8%A8%D8%A7%D8%A8-%D8%A7%D9%84%D9%88%D9%8A%D8%A8/20250203-%D9%85%D8%B5%D8%B1-%D8%AC%D8%A7%D9%85%D8%B9%D8%A9-%D8%A7%D9%84%D8%A3%D8%B2%D9%87%D8%B1-%D8%AA%D8%B9%D9%84%D9%86-%D8%B9%D9%86-%D8%AA%D8%B9%D8%B1%D9%8A%D8%A8-%D8%A7%D9%84%D8%B7%D8%A8
https://www.france24.com/ar/%D9%81%D9%8A%D8%AF%D9%8A%D9%88/20250204-%D8%A5%D8%B3%D8%B1%D8%A7%D8%A6%D9%8A%D9%84-%D8%AA%D8%B3%D8%AA%D8%B9%D8%AF-%D9%84%D8%A5%D8%B1%D8%B3

In [86]:
df_fusionne[df_fusionne.url=='https://www.france24.com/ar/%D9%81%D9%8A%D8%AF%D9%8A%D9%88/20250205-%D8%A7%D9%84%D8%B3%D9%88%D9%8A%D8%AF-%D9%85%D9%82%D8%AA%D9%84-%D9%86%D8%AD%D9%88-%D8%B9%D8%B4%D8%B1%D8%A9-%D8%A3%D8%B4%D8%AE%D8%A7%D8%B5-%D8%A8%D8%A7%D8%B7%D9%84%D8%A7%D9%82-%D9%86%D8%A7%D8%B1-%D9%81%D9%8A-%D9%85%D8%B1%D9%83%D8%B2-%D8%AA%D8%B9%D9%84%D9%8A%D9%85%D9%8A']

,Unnamed: 0,source,title,content,description,url,published_at,tags
243,30,France 24,السويد في حالة صدمة غداة أسوأ عملية قتل في تار...,NaN,NaN,https://www.france24.com/ar/%D9%81%D9%8A%D8%AF...,\n 05 فبراير 2025\n,\n\n أوروبا...


In [126]:
li='https://www.france24.com/ar/%D8%A3%D8%B1%D8%B4%D9%8A%D9%81/2024/12/21-%D8%AF%D9%8A%D8%B3%D9%85%D8%A8%D8%B1-2024'
if(li):
            try:
                driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
                driver.get(li)
                html = driver.page_source
                driver.quit()
                # Parse the page source with BeautifulSoup
                soup2 = BeautifulSoup(html, "html.parser")
                soup2
    
     
                # Extract all news headlines
                contents = soup2.find_all("p", class_="")
                cont = ""
                # Print extracted contents
                for content in contents:
                    cont= cont + "\n"+ content.text.strip()
                print(contents)
                df_fusionne[df_fusionne.url==li].content=cont
                # Extract all news headlines
                description = soup2.find("p", class_="t-content__chapo")
                print(description.text)
                df_fusionne[df_fusionne.url==li].description=description.text
                print(df_fusionne[df_fusionne.url==li].description)
                times = soup2.find("time", class_="")
                df_fusionne[df_fusionne.url==li].published_at=times.text
                tags=soup2.find_all("div", class_="a-tag-section")
                df_fusionne[df_fusionne.url==li].tags=tags[0].text
            except (Exception) as e:
                print(f"Error: {e}")

Error: Message: session not created: This version of ChromeDriver only supports Chrome version 114
Current browser version is 133.0.6943.99 with binary path C:\Program Files\Google\Chrome\Application\chrome.exe
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0048A813+48355]
	(No symbol) [0x0041C4B1]
	(No symbol) [0x00325358]
	(No symbol) [0x003461AC]
	(No symbol) [0x00341EF3]
	(No symbol) [0x00340579]
	(No symbol) [0x00370C55]
	(No symbol) [0x0037093C]
	(No symbol) [0x0036A536]
	(No symbol) [0x003482DC]
	(No symbol) [0x003493DD]
	GetHandleVerifier [0x006EAABD+2539405]
	GetHandleVerifier [0x0072A78F+2800735]
	GetHandleVerifier [0x0072456C+2775612]
	GetHandleVerifier [0x005151E0+616112]
	(No symbol) [0x00425F8C]
	(No symbol) [0x00422328]
	(No symbol) [0x0042240B]
	(No symbol) [0x00414FF7]
	BaseThreadInitThunk [0x762DFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x76F282AE+286]
	RtlGetAppContainerNamedObjectPath [0x76F2827E+238]



In [82]:
description.text

'\n                                                تعيش\xa0السويد\xa0حالة\xa0من\xa0الصدمة\xa0الأربعاء\xa0غداة\xa0أسوأ\xa0عملية\xa0قتل\xa0جماعي\xa0في\xa0تاريخها\xa0راح\xa0ضحيّتها\xa011\xa0شخصا،\xa0من\xa0بينهم\xa0منفذ\xa0الهجوم،\xa0ووقعت\xa0في\xa0مركز\xa0تعليمي\xa0للبالغين\xa0في\xa0أوربرو\xa0(وسط)\xa0مثيرة\xa0العديد\xa0من\xa0الأسئلة\xa0من\xa0دون\xa0إجابات.                    '

In [130]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

options = webdriver.ChromeOptions()
options.add_argument("--headless")

try:
    # Manually set the Chrome version
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager(driver_version="133.0.6943.99").install()), options=options
    )

    li = "https://www.france24.com/ar/%D8%A3%D8%B1%D8%B4%D9%8A%D9%81/2024/12/21-%D8%AF%D9%8A%D8%B3%D9%85%D8%A8%D8%B1-2024"
    driver.get(li)
    html = driver.page_source
    driver.quit()

    soup2 = BeautifulSoup(html, "html.parser")

    contents = soup2.find_all("p", class_="")
    cont = "\n".join(content.text.strip() for content in contents)

    print("Extracted Contents:", cont)

except Exception as e:
    print(f"Error: {e}")


Extracted Contents: 


In [132]:
description = soup2.find("p", class_="t-content__chapo")
print(description.text)

AttributeError: 'NoneType' object has no attribute 'text'

In [10]:
import os
import pandas as pd

# Chemin du dossier contenant les fichiers CSV
dossier_csv = "data"

# Liste pour stocker les DataFrames
dataframes = []

# Lire tous les fichiers CSV contenant "2025" dans le nom
for fichier in os.listdir(dossier_csv):
    if fichier.endswith(".csv") and "2025" in fichier:
        chemin_fichier = os.path.join(dossier_csv, fichier)
        df = pd.read_csv(chemin_fichier)
        dataframes.append(df)

# Fusionner tous les DataFrames
df_fusionne = pd.concat(dataframes, ignore_index=True)

# Enregistrer le résultat dans un nouveau fichier CSV
df_fusionne.to_csv("fichier_fusionne_2025.csv", index=False)

print("Fusion terminée ! Fichier enregistré sous 'fichier_fusionne_2025.csv'.")


Fusion terminée ! Fichier enregistré sous 'fichier_fusionne_2025.csv'.
